In [47]:
import geopandas as gpd 
import descartes
import matplotlib.pyplot as plt
import csv
import nasa_wildfires as fires
import json
import eeconvert
import ee
import requests
import math
import mpmath
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import folium
import requests
import ee
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression #Regresión Lineal con scikit-learn


In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1Adeu5BXvdPFy6CIk9wANx3Fq0bWVGjqeF1Nx36sstRhxtN0Vl6_T6LSapxk

Successfully saved authorization token.


In [3]:
mpmath.mp.dps = 60
def GetSimpleFireSpread(fuelload, fueldepth, windspeed, slope, fuelmoisture, fuelsav):
    # Parameters
    maxval= 0
    if fuelload > 0:
        wo = fuelload # Ovendry fuel loading in (lb/ft^2). Amount of primary prod???
        fd = fueldepth # Fuel depth (ft)
        if fueldepth == 0:
            fd = 0.0000000000000001

        wv = windspeed * 88# Wind velocity at midflame height (ft/minute) = 88 * mph
        fpsa = fuelsav  # Fuel Particle surface area to volume ratio (1/ft)
        mf = fuelmoisture  # Fuel particle moisture content
        h = 8000  # Fuel particle low heat content
        pp = 32.  # Ovendry particle density
        st = 0.0555  # Fuel particle mineral content
        se = 0.010  # Fuel Particle effective mineral content
        mois_ext = 0.12  # Moisture content of extinction or 0.3 if dead
        #calculate slope as degrees
        slope_rad = math.atan(slope)
        slope_degrees = slope_rad / 0.0174533 #radians
        tan_slope = math.tan(slope_rad) #  in radians
        # Betas Packing ratio
        Beta_op = 3.348 * math.pow(fpsa, -0.8189)  # Optimum packing ratio
        ODBD = wo / fd  # Ovendry bulk density
        Beta = ODBD / pp #Packing ratio
        #Beta = 0.00158
        Beta_rel = Beta / Beta_op
        # Reaction Intensity
        WN = wo / (1 + st)  # Net fuel loading
        #A = 1 / (4.774 * pow(fpsa, 0.1) - 7.27)  # Unknown const
        A =  133.0 / math.pow(fpsa, 0.7913) #updated A
        T_max = math.pow(fpsa,1.5) * math.pow(495.0 + 0.0594 * math.pow(fpsa, 1.5),-1.0)  # Maximum reaction velocity
        #T_max = (fpsa*math.sqrt(fpsa)) / (495.0 + 0.0594 * fpsa * math.sqrt(fpsa))
        T = T_max * math.pow((Beta / Beta_op),A) * math.exp(A * (1 - Beta / Beta_op))  # Optimum reaction velocity
        # moisture dampning coefficient
        NM = 1. - 2.59 * (mf / mois_ext) + 5.11 * math.pow(mf / mois_ext, 2.) - 3.52 * math.pow(mf / mois_ext,3.)  # Moisture damping coeff.
        # mineral dampning
        NS = 0.174 * math.pow(se, -0.19)  # Mineral damping coefficient
        #print(T, WN, h, NM, NS)
        RI = T * WN * h * NM * NS
        #RI = 874
        # Propogating flux ratio
        PFR = mpmath.power(192.0 + 0.2595 * fpsa, -1) * mpmath.exp((0.792 + 0.681 * mpmath.sqrt(fpsa)) * (Beta + 0.1))# Propogating flux ratio
        ## Wind Coefficient
        B = 0.02526 * math.pow(fpsa, 0.54)
        C = 7.47 * math.exp(-0.1333 * math.pow(fpsa, 0.55))
        E = 0.715 * math.exp(-3.59 * 10**-4 * fpsa)
        #WC = C * wv**B * math.pow(Beta / Beta_op, -E) #wind coefficient
        if wv > (0.9 * RI): #important - don't know source. Matches BEHAVE
            wv = 0.9 * RI
        WC = (C * wv ** B) * math.pow((Beta / Beta_op), (-E))
        #WC= WC*0.74
        #Slope  coefficient
        SC = 5.275*(Beta**-0.3)*tan_slope**2
        #Heat sink

        EHN = math.exp(-138. / fpsa)  # Effective Heating Number = f(surface are volume ratio)
        QIG = 250. + 1116. * mf  # Heat of preignition= f(moisture content)
        # rate of spread (ft per minute)
        #RI = BTU/ft^2
        numerator = (RI * PFR * (1 + WC + SC))
        denominator = (ODBD * EHN * QIG)
        R = numerator / denominator #WC and SC will be zero at slope = wind = 0
        RT = 384.0/fpsa
        HA = RI*RT
        #fireline intensity as described by Albini via USDA Forest Service RMRS-GTR-371. 2018
        FI = (384.0/fpsa)*RI*(R) ##Uses Reaction Intensity in BTU / ft/ min
        #FI = HA*R
        if (RI <= 0):
            return (maxval, maxval, maxval)
        return (R, RI, FI)
    else:
        return (maxval, maxval, maxval)





In [76]:
coordinatesPrimavera = [[-103.6858, 20.7269],[-103.4552,20.5430]]

# Download a regional GeoJSON of hotspots detected by the MODIS satellite in a recent 7-day period.
def WildfireHotspots():
    data = fires.get_modis(region="central-america", time_range="7d")  
    lat = []
    lon = []
    # Detectar los incendios en el rango de 7 días dentro del rango de la primavera
    for i in range(len(data["features"])):
        if data["features"][i]["geometry"]["coordinates"][0] >= coordinatesPrimavera[0][0] and data["features"][i]["geometry"]["coordinates"][0] <= coordinatesPrimavera[1][0] and data["features"][i]["geometry"]["coordinates"][1] <= coordinatesPrimavera[0][1] and data["features"][i]["geometry"]["coordinates"][1] >= coordinatesPrimavera[1][1]:
            lon.append(data["features"][i]["geometry"]["coordinates"][0])
            lat.append(data["features"][i]["geometry"]["coordinates"][1])
    
    heat_points = list(zip(lon, lat))  # Unificar las coordenadas en tuplas (x, y)
    return heat_points

grid_points = []
grid_points.append(WildfireHotspots())
heat_points=grid_points
heat_points = [(-103.5306,20.7154), (-103.5487,20.7034), (-103.5388,20.7049), (-103.5413,20.6959), (-103.5314,20.6974) ,(-103.5217,20.6988) ,(-103.5298, 20.6883), (-103.52, 20.6898 )]  # Agrega las coordenadas de los puntos de calor aquí



In [74]:
def distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def closest_point(p, points):
    min_distance = float('inf')
    for i, row in points.iterrows():
        dist = distance(p[0], p[1], row.geometry.centroid.x, row.geometry.centroid.y)
        if dist < min_distance:
            closest = (row.geometry.centroid.y, row.geometry.centroid.x)
            min_distance = dist
    return closest


In [75]:
# Carga del shapefile
gdf = gpd.read_file(r'D:\Escritorio\Modular\Primavera.shp')

# Definición de la cuadrícula
xmin, ymin, xmax, ymax = -103.6858, 20.5430, -103.4552, 20.7269
grid_size = 0.01
cols = list(np.arange(xmin, xmax, grid_size))
rows = list(np.arange(ymin, ymax, grid_size))
rows.reverse()

polygons = []
for x in cols:
    for y in rows:
        polygons.append(Polygon([(x,y), (x+grid_size,y), (x+grid_size,y-grid_size), (x,y-grid_size)]))

grid = gpd.GeoDataFrame({'geometry':polygons})
intersected = gpd.overlay(gdf, grid, how='intersection')

# Calcular los puntos más cercanos a los polígonos de la cuadrícula
closest_points = []
for p in heat_points:
    closest_points.append(closest_point(p, intersected))
    
# Creación de un mapa de folium con el shapefile y la cuadrícula
m = folium.Map(location=[20.6, -103.57], zoom_start=12)

# Agregar shapefile al mapa
folium.GeoJson(gdf).add_to(m)

for index, row in intersected.iterrows():
    tooltip = f"{index}"
    y_centrum, x_centrum = row.geometry.centroid.y, row.geometry.centroid.x
    if (y_centrum, x_centrum) in closest_points:
        marker =  folium.Marker([y_centrum, x_centrum], 
                  icon=folium.Icon(color='red', icon='pushpin'))
    else:
        marker = folium.Marker([y_centrum, x_centrum], tooltip=tooltip)
    marker.add_to(m)
    

# Mostrar el mapa
m

C:\Users\chiva\AppData\Local\Temp\ipykernel_11104\1220589276.py:17: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  intersected = gpd.overlay(gdf, grid, how='intersection')


In [6]:
pip install geemap

In [17]:
# Definir la URL de la API de OpenWeatherMap y los parámetros de consulta
url = "https://api.openweathermap.org/data/2.5/weather"

# Definir las listas de las variables
wind_speed = []
fuel_depth=[]
temp = []
wind_direction = []
slope = []
fuel_moisture = []
fuelload = []
coordenadas = []


# Establecer los parámetros de la consulta
fecha_inicio = '2023-01-01'
fecha_fin = '2023-12-31'

# Consulta de datos de vegetación
coleccion = ee.ImageCollection('MODIS/006/MOD13A2').filterDate(fecha_inicio, fecha_fin)

# Load the Global Forest Change dataset
dataset = ee.Image('UMD/hansen/global_forest_change_2019_v1_7')

# Select the tree cover band
treeCover = dataset.select('treecover2000')

# Iterar sobre los registros
for i, row in intersected.iterrows():
    
    y_centrum, x_centrum = row.geometry.centroid.y, row.geometry.centroid.x
    coordenadas.append([x_centrum, y_centrum])
    
    print(f"Coordenadas del cuadrante {i}: {x_centrum},{y_centrum}")
    
    region = ee.Geometry.Point(coordenadas[i])
   # Define your region of interest (e.g., a point or a geometry) 
    roi = ee.Geometry.Point(coordenadas[i])
    
    params = {
        "lat": y_centrum,  # Latitud
        "lon": x_centrum,  # Longitud
        "appid": "88ea1e1a48e480178b9c4ab177c2475f",  # Tu ID de API de OpenWeatherMap
        "units": "metric"  # Unidades métricas (para obtener la velocidad del viento en m/s)
    }
    
    # Hacer una solicitud HTTP a la API y obtener la respuesta JSON
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
    
        # Extraer la velocidad del viento
        wind_speed.append(data["wind"]["speed"])
        temp.append(data["main"]["temp"])
        wind_direction.append(data["wind"]["deg"])
        
        imagen = coleccion.select('NDVI').mean().clip(region)

        # Obtener una región de interés como una imagen en forma de píxeles
        datos_imagen = imagen.reduceRegion(reducer=ee.Reducer.mean(), geometry=region, scale=20)

        # Obtener el valor promedio del NDVI en la región de interés
        valor_ndvi = datos_imagen.get('NDVI')
        
        
        
        # Extraer el valor del NDVI
        valor_ndvi = ee.Number(valor_ndvi)

        # Estimar la masa de vegetación en lb/ft^2
        slope_ = 0.5  # Pendiente de la relación lineal
        intercept = 0.2  # Término de intercepción de la relación lineal

        vegetation_mass = valor_ndvi.multiply(slope_).add(intercept)

        # Imprimir el resultado
       # print(vegetation_mass.getInfo())
        
        

        fuelload.append(vegetation_mass.getInfo())
        
        
        # Calculate the mean tree cover within the region of interest
        treeCover_mean = treeCover.reduceRegion(ee.Reducer.mean(), roi, 30)

        # Get the tree cover value
        treeCoverValue = ee.Number(treeCover_mean.get('treecover2000'))

        # Convert tree cover from percentage to feet
        conversion_factor = 43.560  # 1 acre = 43,560 square feet
        treeCoverValue_feet = treeCoverValue.multiply(conversion_factor)
        



        fuel_depth.append(treeCoverValue_feet.getInfo())
      
        

        # Extraer la altitud y la presión atmosférica para calcular la pendiente de la respuesta JSON
        altitude = None
        atmospheric_pressure = data["main"]["pressure"]  # En hPa
        try:
            altitude = data["main"]["sea_level"]  # En metros
            slope.append(0.102 * altitude / atmospheric_pressure)  # En grados
        except KeyError:
            slope.append(0)
        
        # Calcular la humedad de combustible utilizando la fórmula de Deeming
        temp_calculate = data["main"]["temp"] - 273.15  # Convertir de Kelvin a Celsius
        relative_moisture = data["main"]["humidity"]
        precipitation = data.get("rain", {}).get("1h", 0)  # Obtener la precipitación de los últimos 60 minutos (si está disponible)
        wind_speed_calculate = data["wind"]["speed"]

        fuel_moisture.append(20 + 280 / (temp_calculate + 273) - relative_moisture + 0.1 * wind_speed_calculate + 0.2 * precipitation)
    else:
        print('La solicitud falló con el código de estado:', response.status_code)

        

def replace_zero_with_average(arr):
    # Calcula el promedio de todos los registros del arreglo
    total_sum = sum(arr)
    non_zero_count = len(arr) - arr.count(0)
    average = total_sum / non_zero_count if non_zero_count > 0 else 0

    # Recorre el arreglo y reemplaza los valores iguales a 0 por el promedio
    for i in range(len(arr)):
        if arr[i] == 0:
            arr[i] = average

    return arr

# Ejemplo de uso
fuel_depth = replace_zero_with_average(fuel_depth)
# print(fuel_depth)

# Imprimir los resultados
print(f"Velocidad del viento: {wind_speed} m/s")
print(f"Pendiente: {slope} grados")
print(f"Humedad del combustible: {fuel_moisture}%")
print("Temperatura:", temp, "°C")
print("Dirección del viento:", wind_direction, "°")
print("FuelLoad:", fuelload)
print("FuelDepth:", fuel_depth)


Coordenadas del cuadrante 0: -103.6778264314601,20.694401751245415
Coordenadas del cuadrante 1: -103.68006778254957,20.687856892589483
Coordenadas del cuadrante 2: -103.67655120763247,20.68240892258042
Coordenadas del cuadrante 3: -103.6712522399255,20.69459776664458
Coordenadas del cuadrante 4: -103.6708,20.68800000000002
Coordenadas del cuadrante 5: -103.67026502391904,20.67833076360917
Coordenadas del cuadrante 6: -103.66893476420363,20.670865830756377
Coordenadas del cuadrante 7: -103.66670327192554,20.655164030824036
Coordenadas del cuadrante 8: -103.66680854297383,20.647389881031476
Coordenadas del cuadrante 9: -103.66800223901635,20.637585470546636
Coordenadas del cuadrante 10: -103.66847982232078,20.628081125705844
Coordenadas del cuadrante 11: -103.6675041207906,20.618680236251144
Coordenadas del cuadrante 12: -103.66654456175115,20.611950987378673
Coordenadas del cuadrante 13: -103.66062918482406,20.69590599248074
Coordenadas del cuadrante 14: -103.66086961774421,20.687916857

Coordenadas del cuadrante 121: -103.59079999999993,20.588000000000005
Coordenadas del cuadrante 122: -103.59078082083074,20.578191060151745
Coordenadas del cuadrante 123: -103.58708941358844,20.57256314020816
Coordenadas del cuadrante 124: -103.58146564934059,20.71504838792559
Coordenadas del cuadrante 125: -103.58079999999997,20.708000000000027
Coordenadas del cuadrante 126: -103.58079999999997,20.698000000000025
Coordenadas del cuadrante 127: -103.58079999999997,20.68800000000002
Coordenadas del cuadrante 128: -103.58079999999997,20.67800000000002
Coordenadas del cuadrante 129: -103.58079999999997,20.668000000000024
Coordenadas del cuadrante 130: -103.58079999999997,20.65800000000002
Coordenadas del cuadrante 131: -103.58079999999997,20.648000000000017
Coordenadas del cuadrante 132: -103.58079999999997,20.638000000000012
Coordenadas del cuadrante 133: -103.58079999999997,20.628000000000014
Coordenadas del cuadrante 134: -103.58079999999997,20.61800000000001
Coordenadas del cuadrante 

Coordenadas del cuadrante 239: -103.52079999999991,20.60800000000001
Coordenadas del cuadrante 240: -103.52079999999991,20.59800000000001
Coordenadas del cuadrante 241: -103.52079999999991,20.588000000000005
Coordenadas del cuadrante 242: -103.52079999999991,20.578000000000003
Coordenadas del cuadrante 243: -103.52079999999991,20.56800000000001
Coordenadas del cuadrante 244: -103.51990902206863,20.559257547592797
Coordenadas del cuadrante 245: -103.51587093941438,20.548584361299355
Coordenadas del cuadrante 246: -103.51080687982692,20.717993022950125
Coordenadas del cuadrante 247: -103.5107999999999,20.708000000000027
Coordenadas del cuadrante 248: -103.5107999999999,20.698000000000025
Coordenadas del cuadrante 249: -103.5107999999999,20.68800000000002
Coordenadas del cuadrante 250: -103.5107999999999,20.67800000000002
Coordenadas del cuadrante 251: -103.5107999999999,20.668000000000024
Coordenadas del cuadrante 252: -103.5107999999999,20.65800000000002
Coordenadas del cuadrante 253: -

PRUEBAS

In [84]:
# Definir la URL de la API de OpenWeatherMap y los parámetros de consulta
url = "https://history.openweathermap.org/data/2.5/history/city"
# Definir las listas de las variables
wind_speed = []
fuel_depth=[]
temp = []
wind_direction = []
slope = []
fuel_moisture = []
fuelload = []
coordenadas = []

# Establecer los parámetros de la consulta
fecha_inicio = '2023-01-01'
fecha_fin = '2023-12-31'

# Consulta de datos de vegetación
coleccion = ee.ImageCollection('MODIS/006/MOD13A2').filterDate(fecha_inicio, fecha_fin)

# Load the Global Forest Change dataset
dataset = ee.Image('UMD/hansen/global_forest_change_2019_v1_7')

# Select the tree cover band
treeCover = dataset.select('treecover2000')
# Iterar sobre los registros
for i, row in intersected.iterrows():
    
    y_centrum, x_centrum = row.geometry.centroid.y, row.geometry.centroid.x
    
    
    coordenadas.append([x_centrum, y_centrum])
    print(f"Coordenadas del cuadrante {i}: {x_centrum},{y_centrum}")
    
    region = ee.Geometry.Point(coordenadas[i])
   # Define your region of interest (e.g., a point or a geometry) 
    roi = ee.Geometry.Point(coordenadas[i])
    
    params = {
        "lat": y_centrum,  # Latitud
        "lon": x_centrum,  # Longitud
        "type": "hour",
        "start": 1661403600,  # Fecha de inicio en formato UNIX timestamp
        "end": 1661404320,  # Fecha de fin en formato UNIX timestamp
        "appid": "bd5e378503939ddaee76f12ad7a97608"  # Tu ID de API de OpenWeatherMap
    }

    # Hacer una solicitud HTTP a la API y obtener la respuesta JSON
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
    
        # Extraer la velocidad del viento
        wind_speed.append(data["wind"]["speed"])
        temp.append(data["main"]["temp"])
        wind_direction.append(data["wind"]["deg"])
        
        imagen = coleccion.select('NDVI').mean().clip(region)

        # Obtener una región de interés como una imagen en forma de píxeles
        datos_imagen = imagen.reduceRegion(reducer=ee.Reducer.mean(), geometry=region, scale=20)

        # Obtener el valor promedio del NDVI en la región de interés
        valor_ndvi = datos_imagen.get('NDVI')
         
        
        # Extraer el valor del NDVI
        valor_ndvi = ee.Number(valor_ndvi)

        # Estimar la masa de vegetación en lb/ft^2
        slope_ = 0.5  # Pendiente de la relación lineal
        intercept = 0.2  # Término de intercepción de la relación lineal

        vegetation_mass = valor_ndvi.multiply(slope_).add(intercept)

        # Imprimir el resultado
       # print(vegetation_mass.getInfo())
        

        fuelload.append(vegetation_mass.getInfo())
        
        
        # Calculate the mean tree cover within the region of interest
        treeCover_mean = treeCover.reduceRegion(ee.Reducer.mean(), roi, 30)

        # Get the tree cover value
        treeCoverValue = ee.Number(treeCover_mean.get('treecover2000'))

        # Convert tree cover from percentage to feet
        conversion_factor = 43.560  # 1 acre = 43,560 square feet
        treeCoverValue_feet = treeCoverValue.multiply(conversion_factor)
        

        fuel_depth.append(treeCoverValue_feet.getInfo())
         

        # Extraer la altitud y la presión atmosférica para calcular la pendiente de la respuesta JSON
        altitude = None
        atmospheric_pressure = data["main"]["pressure"]  # En hPa
        try:
            altitude = data["main"]["sea_level"]  # En metros
            slope.append(0.102 * altitude / atmospheric_pressure)  # En grados
        except KeyError:
            slope.append(0)
        
        # Calcular la humedad de combustible utilizando la fórmula de Deeming
        temp_calculate = data["main"]["temp"] - 273.15  # Convertir de Kelvin a Celsius
        relative_moisture = data["main"]["humidity"]
        precipitation = data.get("rain", {}).get("1h", 0)  # Obtener la precipitación de los últimos 60 minutos (si está disponible)
        wind_speed_calculate = data["wind"]["speed"]

        fuel_moisture.append(20 + 280 / (temp_calculate + 273) - relative_moisture + 0.1 * wind_speed_calculate + 0.2 * precipitation)
    else:
        print('La solicitud falló con el código de estado:', response.status_code)

Coordenadas del cuadrante 0: -103.6778264314601,20.694401751245415
La solicitud falló con el código de estado: 401
Coordenadas del cuadrante 1: -103.68006778254957,20.687856892589483
La solicitud falló con el código de estado: 401
Coordenadas del cuadrante 2: -103.67655120763247,20.68240892258042
La solicitud falló con el código de estado: 401
Coordenadas del cuadrante 3: -103.6712522399255,20.69459776664458
La solicitud falló con el código de estado: 401
Coordenadas del cuadrante 4: -103.6708,20.68800000000002
La solicitud falló con el código de estado: 401
Coordenadas del cuadrante 5: -103.67026502391904,20.67833076360917
La solicitud falló con el código de estado: 401
Coordenadas del cuadrante 6: -103.66893476420363,20.670865830756377
La solicitud falló con el código de estado: 401
Coordenadas del cuadrante 7: -103.66670327192554,20.655164030824036
La solicitud falló con el código de estado: 401
Coordenadas del cuadrante 8: -103.66680854297383,20.647389881031476
La solicitud falló c

KeyboardInterrupt: 

In [10]:
import requests
import ee
import geemap

# Definir la URL de la API de OpenWeatherMap y los parámetros de consulta
url = "https://history.openweathermap.org/data/2.5/history/city"

# Definir las listas de las variables
wind_speed = []
fuel_depth=[]
temp = []
wind_direction = []
slope = []
fuel_moisture = []
fuelload = []
coordenadas = []




# Establecer los parámetros de la consulta
fecha_inicio = '2023-01-01'
fecha_fin = '2023-12-31'

# Consulta de datos de vegetación
coleccion = ee.ImageCollection('MODIS/006/MOD13A2').filterDate(fecha_inicio, fecha_fin)

# Load the Global Forest Change dataset
dataset = ee.Image('UMD/hansen/global_forest_change_2019_v1_7')

# Select the tree cover band
treeCover = dataset.select('treecover2000')
start = 1557742066
end = 1557745666
# Iterar sobre los registros
for i, row in intersected.iterrows():
    
    y_centrum, x_centrum = row.geometry.centroid.y, row.geometry.centroid.x
    

    
    coordenadas.append([x_centrum, y_centrum])
    
    print(f"Coordenadas del cuadrante {i}: {x_centrum},{y_centrum}")
    
    region = ee.Geometry.Point(coordenadas[i])
   # Define your region of interest (e.g., a point or a geometry) 
    roi = ee.Geometry.Point(coordenadas[i])
    
    params = {
        "lat": 20.6912,
        "lon": -103.5104,
        "type": "hour",
        "start": start,
        "end": end,
        "appid": "d1100fa8219b246da4eff85803546c71"
    }
    
    # Hacer una solicitud HTTP a la API y obtener la respuesta JSON
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        print(data)
        # Extraer la velocidad del viento
        wind_speed.append(data["wind"]["speed"])
        temp.append(data["main"]["temp"])
        wind_direction.append(data["wind"]["deg"])
        
        imagen = coleccion.select('NDVI').mean().clip(region)

        # Obtener una región de interés como una imagen en forma de píxeles
        datos_imagen = imagen.reduceRegion(reducer=ee.Reducer.mean(), geometry=region, scale=20)

        # Obtener el valor promedio del NDVI en la región de interés
        valor_ndvi = datos_imagen.get('NDVI')
        
        
        
        # Extraer el valor del NDVI
        valor_ndvi = ee.Number(valor_ndvi)

        # Estimar la masa de vegetación en lb/ft^2
        slope_ = 0.5  # Pendiente de la relación lineal
        intercept = 0.2  # Término de intercepción de la relación lineal

        vegetation_mass = valor_ndvi.multiply(slope_).add(intercept)

        # Imprimir el resultado
       # print(vegetation_mass.getInfo())
        
        

        fuelload.append(vegetation_mass.getInfo())
        
        
        # Calculate the mean tree cover within the region of interest
        treeCover_mean = treeCover.reduceRegion(ee.Reducer.mean(), roi, 30)

        # Get the tree cover value
        treeCoverValue = ee.Number(treeCover_mean.get('treecover2000'))

        # Convert tree cover from percentage to feet
        conversion_factor = 43.560  # 1 acre = 43,560 square feet
        treeCoverValue_feet = treeCoverValue.multiply(conversion_factor)
        



        fuel_depth.append(treeCoverValue_feet.getInfo())
      
        

        # Extraer la altitud y la presión atmosférica para calcular la pendiente de la respuesta JSON
        altitude = None
        atmospheric_pressure = data["main"]["pressure"]  # En hPa
        try:
            altitude = data["main"]["sea_level"]  # En metros
            slope.append(0.102 * altitude / atmospheric_pressure)  # En grados
        except KeyError:
            slope.append(0)
        
        # Calcular la humedad de combustible utilizando la fórmula de Deeming
        temp_calculate = data["main"]["temp"] - 273.15  # Convertir de Kelvin a Celsius
        relative_moisture = data["main"]["humidity"]
        precipitation = data.get("rain", {}).get("1h", 0)  # Obtener la precipitación de los últimos 60 minutos (si está disponible)
        wind_speed_calculate = data["wind"]["speed"]

        fuel_moisture.append(20 + 280 / (temp_calculate + 273) - relative_moisture + 0.1 * wind_speed_calculate + 0.2 * precipitation)
    else:
        print('La solicitud falló con el código de estado:', response.status_code)

        

def replace_zero_with_average(arr):
    # Calcula el promedio de todos los registros del arreglo
    total_sum = sum(arr)
    non_zero_count = len(arr) - arr.count(0)
    average = total_sum / non_zero_count if non_zero_count > 0 else 0

    # Recorre el arreglo y reemplaza los valores iguales a 0 por el promedio
    for i in range(len(arr)):
        if arr[i] == 0:
            arr[i] = average

    return arr

# Ejemplo de uso
fuel_depth = replace_zero_with_average(fuel_depth)
# print(fuel_depth)

# Imprimir los resultados
print(f"Velocidad del viento: {wind_speed} m/s")
print(f"Pendiente: {slope} grados")
print(f"Humedad del combustible: {fuel_moisture}%")
print("Temperatura:", temp, "°C")
print("Dirección del viento:", wind_direction, "°")
print("FuelLoad:", fuelload)
print("FuelDepth:", fuel_depth)


NameError: name 'propagation_gdf' is not defined

OTRAS PRUEBAS

In [55]:
# Coordenadas de los puntos de calor de referencia
heat_points = [(-103.5306,20.7154), (-103.5487,20.7034), (-103.5388,20.7049), (-103.5413,20.6959), (-103.5314,20.6974) ,(-103.5217,20.6988) ,(-103.5298, 20.6883), (-103.52, 20.6898 )]  # Agrega las coordenadas de los puntos de calor aquí

# Calcular el índice de propagación del fuego para los puntos de calor de referencia
I = []

for i, row in intersected.iterrows():
    fuelload_val = fuelload[i]
    fuel_depth_val = fuel_depth[i]
    fuelsav = 3500
    slope_val = slope[i] # Pendiente en grados
    wind_speed_val = wind_speed[i] * 2.237 # Velocidad del viento en m/s
    wind_direction_val = wind_direction[i]
    temperature_val = temp[i]
    fuel_moisture_val = fuel_moisture[i] / 1000 # Humedad de combustible
    
    # Calcular el índice de propagación del fuego para cada punto de calor de referencia
    point_indices = []
    for heat_point in heat_points:
        heat_x, heat_y = heat_point
        distance_to_heat = math.sqrt((row.geometry.centroid.x - heat_x)**2 + (row.geometry.centroid.y - heat_y)**2)
    
        # Calcular la influencia basada en la intensidad del fuego en los heat points (puedes ajustar el factor)
        fire_intensity = 100  # Aquí puedes usar la intensidad real del fuego en el punto de calor
        if distance_to_heat != 0:
            influence_factor = fire_intensity / distance_to_heat
        else:
            pass
        point_indices.append(GetSimpleFireSpread(fuelload_val, fuel_depth_val, wind_speed_val, slope_val, fuel_moisture_val, fuelsav)[0] * influence_factor)
    # Calcular el índice de propagación del fuego para el cuadrante actual
    I_val = sum(point_indices)
    I.append(I_val)

# Crear un nuevo GeoDataFrame con el índice de propagación del fuego para cada cuadrante
intersected["I"] = I

# Crear un mapa de folium con el GeoDataFrame y los puntos de calor
mapa = folium.Map(location=[20.6, -103.57], zoom_start=12)

# Convertir los datos en intersected['I'] a tipo float
intersected['I'] = intersected['I'].astype(float)

# Crear una función que asigna el color adecuado a cada valor del índice de propagación
colormap = cm.LinearColormap(colors=['green', 'yellow', 'orange', 'red'], vmin=intersected['I'].min(), vmax=intersected['I'].max())

# Crear el objeto de Folium
geojson = folium.GeoJson(intersected, name='Indice de propagacion del fuego', 
                         tooltip=folium.features.GeoJsonTooltip(fields=['I'], 
                                                                 labels=True, 
                                                                 sticky=False),
                         style_function=lambda feature: {
                             'fillColor': colormap(feature['properties']['I']),
                             'color': 'black',
                             'weight': 1,
                             'fillOpacity': 0.7
                         })

# Agregar el objeto GeoJson al mapa de Folium
geojson.add_to(mapa)

# Agregar marcadores morados para los puntos de calor
for heat_point in heat_points:
    folium.Marker([heat_point[1], heat_point[0]], 
                  icon=folium.Icon(color='purple', icon='pushpin')).add_to(mapa)

# Agregar la leyenda al mapa de Folium
colormap.caption = 'Indice de propagacion del fuego'
colormap.add_to(mapa)

# Mostrar el mapa de Folium
mapa

In [51]:
def regresion_lineal(nueva_hora, horas_heat_points, longitud_heat_point, latitud_heat_point):
    #Ajusta
    regresion_lineal = LinearRegression() # creamos una instancia de LinearRegression
    # instruimos a la regresión lineal que aprenda de los datos (x,y)
    regresion_lineal.fit(horas_heat_points.reshape(-1,1), longitud_heat_point) 
    # vemos los parámetros que ha estimado la regresión lineal
    # print('w = ' + str(regresion_lineal.coef_) + ', b = ' + str(regresion_lineal.intercept_))
    # plt.plot(x, regresion_lineal.predict(x.reshape(-1,1)), color='red')

    # vamos a predecir
    # nueva_hora = np.array([1600]) 
    longitud = regresion_lineal.predict(nueva_hora.reshape(-1,1))
    # print(longitud)
    # plt.figure()

    #Datos
    # hacemos un gráfico de los datos que hemos generado
    # plt.scatter(x,y2,label='y2', color='blue')
    # plt.title('latitud')


    #Ajusta
    regresion_lineal = LinearRegression() # creamos una instancia de LinearRegression
    # instruimos a la regresión lineal que aprenda de los datos (x,y)
    regresion_lineal.fit(horas_heat_points.reshape(-1,1), latitud_heat_point) 
    # vemos los parámetros que ha estimado la regresión lineal
    # print('w = ' + str(regresion_lineal.coef_) + ', b = ' + str(regresion_lineal.intercept_))
    # resultado: w = [0.09183522], b = 1.2858792525736682
    # plt.plot(x, regresion_lineal.predict(x.reshape(-1,1)), color='red')

    # vamos a predecir y = regresion_lineal(5)
    # nueva_hora = np.array([1600]) 
    latitud = regresion_lineal.predict(nueva_hora.reshape(-1,1))
    return latitud[0], longitud[0]

In [72]:
horas_heat_points = np.array([507,607,707,807,907,1007,1107,1207,1307,1407]) # horas

longitud_heat_point = np.array([-103.5306,-103.5487,-103.5388,-103.5413,-103.5314,-103.5217,-103.5298,-103.52  ,-103.5104,-103.5166]) # longitud

latitud_heat_point = np.array([20.7154,20.7034,20.7049,20.6959,20.6974,20.6988,20.6883,20.6898,20.6912,20.6717]) # latitud

hora1 = np.array([1607])
hora2 = np.array([1907])
hora3 = np.array([2207])

horas = []
horas.insert(len(horas),hora1)
horas.insert(len(horas),hora2)
horas.insert(len(horas),hora3)
for i in horas:
    prediction_y , prediction_x = regresion_lineal(i, horas_heat_points,  longitud_heat_point, latitud_heat_point)
    np.append(longitud_heat_point, prediction_x)
    np.append(latitud_heat_point, prediction_y)
    folium.Marker([prediction_y, prediction_x], icon=folium.Icon(color='red', icon='pushpin')).add_to(mapa)
mapa

20.67300484848485 -103.50796060606059
20.662539393939397 -103.4982824242424
20.652073939393944 -103.48860424242423
